# Genereate Longest Lines for Polygons

## Import packages

In [1]:
import os
import pandas as pd
from arcgis.gis import GIS
import arcgis

## Extract geometry objects from polygons

The value for the *polygon* variable should be replaced with your polygon feature class file path.

In [2]:
polygons = "NC_Counties"

src = arcpy.da.SearchCursor(polygons, ["Shape@", "OID@"])
polygonList = []
for row in src:
    polygonList.append(row[0])
del src
print(f"There are {len(polygonList)} features to process")

There are 2 features to process


## Create a function for processing

This function iterates through all the polygons provided. It calculates the distance between every coordinate pair in the polygon. It stores the coordinates and distance of the longest line.

In [3]:
def LongestLineInFeature(polygon):
    pointList = polygonList[polygon][0]
    i = 0
    j = 0
    numPnts = len(pointList)
    print(f"There are {numPnts} verticies to process")
    print(f"Processing feature #{polygon}")
    longestDistance = 0
    longestDistanceCoords = []
    for point in pointList:
        i = 0
        while i != numPnts:
            pnt1 = arcpy.PointGeometry(pointList[i])
            pnt2 = arcpy.PointGeometry(pointList[j])
            dist = pnt1.distanceTo(pnt2)
            if dist > longestDistance:
                longestDistance = dist
                longestDistanceCoords = [pnt1, pnt2]
            i += 1
        j += 1
    print(f"The longest line distance is {longestDistance}")
    return [longestDistance, longestDistanceCoords]

## Use the function and generate output

This code creates a feature class in the project's default geodatabase called *LongestLines*. It adds the longest line as a feature to this feature class.

You can replace the values for the *outLocation* and *outName* variables to change the output location.

Depending on the complexity of your polygons, this code can take a long time to run.

In [4]:
arcpy.overwriteOutput = True

outLocation = arcpy.env.workspace
outName = "LongestLines"
fcPath = os.path.join(outLocation, outName)

arcpy.management.CreateFeatureclass(outLocation, outName, "POLYLINE", spatial_reference=3857)
arcpy.management.AddField(fcPath, "LongestDistance", "DOUBLE")

iCur = arcpy.da.InsertCursor(fcPath, ['Shape@', 'LongestDistance'])

for num in range(len(polygonList)):
    print(num)
    output = LongestLineInFeature(num)
    coords = arcpy.Array()
    for coord in output[1]:
        coords.append(coord.centroid)
    polyline = arcpy.Polyline(coords)
    iCur.insertRow([polyline, output[0]])
del iCur

0
There are 340 verticies to process
Processing feature #0
The longest line distance is 64629.578825363526
1
There are 225 verticies to process
Processing feature #1
The longest line distance is 73768.03327788014


## Visualize results

Use the ArcGIS API for Python to embed a web map, and display the results.
You will want to update the location the map widget shows using the parameters for the gis.map function.

In [5]:
gis = GIS()
map1 = gis.map('Mebane, NC', zoomlevel = 9)
sdf = pd.DataFrame.spatial.from_featureclass(fcPath)
map1.add_layer(sdf)
map1

MapView(layout=Layout(height='400px', width='100%'))